# Notes on SGP4 Libraries and Comparisons of their Propagators

# Some History
SGP4 is the propagator used strictly with the publicly available TLEs. The original code by the author is provided primarily at the website [here](https://celestrak.org/software/vallado-sw.php), but I have collated a copy in my repository [here](https://github.com/icyveins7/sgp4-download-celestrak) along with some notes about the two different versions available on the website. To make things clear, I will reference the versions of the code I reupload onto my repository: current test is at commit 09142b1.

This current repository, ```pydsproutines```, also contains some code from the python libraries ```sgp4``` (which is a simple wrapper around the SGP4 library) and ```skyfield``` (which has more functions, and itself wraps the python ```sgp4``` library).

# Comparisons
We would like to compare the results of the same propagation in the above libraries, bearing in mind that the calling structure is different; if the inputs required are different, we would like to determine what is required to make the results the same, if any.

# Coordinate Systems
Before we start, it is important to understand the different coordinate systems at play here. The TLEs and the SGP4 propagator are closely intertwined, and both are expected to operate _solely_ in what is known as the __TEME (True Equator Mean Equinox)__ coordinate system.

However, for most intents and purposes, we require the __ITRF (International Terrestrial Reference Frame)__, which is a specification of an __ECEF (Earth-Centered Earth-Fixed)__ frame. This conversion from TEME to ITRF is _non-trivial_, and is the part which requires a lot of external input i.e. not just the TLE, but other things known as Earth Orientation Parameters are required.

See [this reference](https://apps.dtic.mil/sti/pdfs/ADA637370.pdf) for a great discussion on the different coordinate frames.

## Test 1: Compare pure SGP4 propagation in the TEME frame.
1. Python + sgp4
2. Python + skyfield
3. C++ original function

We assume the 'wgs84' constants in all cases where applicable. We will use the following TLE:

In [1]:
line1 = "1 19548U 88091B   23110.52047465 -.00000310  00000+0  00000+0 0  9996"
line2 = "2 19548  13.4884 348.9767 0037807 339.0721  18.7948  1.00270407113813"

import numpy as np

# Parameters of propagation
year = 2023
mon = 4
day = 21
hr = 12
minute = 0
sec = 0

# Test 1.1, Python + sgp4
from sgp4.api import Satrec, jday
from sgp4.api import WGS72, WGS84

sat = Satrec.twoline2rv(line1, line2, WGS84)
jd, fr = jday(year, mon, day, hr, minute, sec)
e, r, v = sat.sgp4(jd, fr)
print(r, v)

# Test 1.2 Python + skyfield
from skyfield.api import EarthSatellite, load
from skyfield.sgp4lib import TEME

sfsat = EarthSatellite(line1, line2)
ts = load.timescale()
t = ts.utc(year, mon, day, hr, minute, sec)
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)
print(pos.km, vel.km_per_s)

print("Error = %g" % (np.linalg.norm(np.array(r) - np.array(pos.km))))

(39641.16403399667, -13833.063014725132, -1428.375749762684) (1.0143747638499885, 2.8263194734421773, 0.7116770873234988)
[ 39641.17161091 -13833.0751021   -1428.37827126] [1.01437532 2.82632002 0.71167724]
Error = 0.014487


Clearly, there is a slight difference here, on the order of about 10m (0.01km). This is more than likely to be good enough, but why is it different if ```skyfield``` is only wrapping ```sgp4```? It turns out there's a simple explanation for this, which is pretty obvious looking at the code; ```skyfield``` defaults to using WGS72, and doesn't have an option to specify this in ```EarthSatellite()``` directly.

Note that it is _not at all obvious_ that the wrapper function ```Satrec.twoline2rv()``` takes in an optional parameter for the gravitational constant. It is apparent if you have looked at the original C++ code, which has an argument ```gravconsttype whichconst```, but otherwise since the python ```sgp4``` library wraps it and exposes an interface without good docstrings, the only documentation that shows this is at the PyPi site [here](https://pypi.org/project/sgp4/), scrolling down to the 'Gravity' section. Notably, it does claim that using WGS84 at this point in the propagator is likely to perform worse when compared to real measurements of satellite positions, due to the TLEs being generated with WGS72 models in mind.

In [2]:
# Test 1.1, Python + sgp4
from sgp4.api import Satrec, jday
from sgp4.api import WGS72, WGS84

sat = Satrec.twoline2rv(line1, line2, WGS72)
jd, fr = jday(year, mon, day, hr, minute, sec)
e, r, v = sat.sgp4(jd, fr)
print(r, v)

# Test 1.2 Python + skyfield
from skyfield.api import EarthSatellite, load
from skyfield.sgp4lib import TEME

sfsat = EarthSatellite(line1, line2)
ts = load.timescale()
t = ts.utc(year, mon, day, hr, minute, sec)
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)
print(pos.km, vel.km_per_s)

print("Error = %g" % (np.linalg.norm(np.array(r) - np.array(pos.km))))

(39641.171610908845, -13833.075102099956, -1428.3782712556995) (1.0143753225229308, 2.826320024639999, 0.7116772422656449)
[ 39641.17161091 -13833.0751021   -1428.37827126] [1.01437532 2.82632002 0.71167724]
Error = 2.12967e-10


In [3]:
# Test 1.3 C++ original library
# Note that the original SGP4 twoline2rv function requires many more inputs, of which most are unnecessary.
# A good way to determine what to use is to look at what the python-sgp4 library did:
# SGP4Funcs::twoline2rv(line1, line2, ' ', ' ', 'i', whichconst,
#                          dummy, dummy, dummy, self->satrec);
# We see that the first two char options are unnecessary, and hence any dummy char can be placed. 'i' is to use the improved mode vs the older mode.
# The following 3 dummy variables are unused as we perform the propagation ourselves later on.
# Notably, the original code's propagator, sgp4(), only accepts time since epoch as the input.
# This means we have to find the difference between epoch time and the target time; this can be done by referencing the
# struct's 'jdsatepoch' & 'jdsatepochF' variables.
# Again, we can look at what the python-sgp4 library did:
# double tsince = (jd - satrec.jdsatepoch) * 1440.0
#                  + (fr - satrec.jdsatepochF) * 1440.0;
#
#
# Output (copied from my externally compiled and tested, you're gonna have to trust i did the above):
cpp_x = np.array([39641.171610909, -13833.075102099, -1428.378271256])
cpp_v = np.array([1.014375323, 2.826320025, 0.711677242])

print("Error = %g" % (np.linalg.norm(cpp_x - np.array(r))))


Error = 1.01447e-09


Alright, so far so good! Now we move on to the next task..

## Test 2: Converting TEME to ITRF
1. Python + skyfield
2. C++ original function

Note that the python ```sgp4``` library does not contain the coordinate transformation functions; this is only provided in ```skyfield```. This accurately reflects the original code, as the ```sgp4()``` function in C++ is contained in a separate header and source file (and even in a separate folder) from the coordinate transformations, found in the ```AstroLib``` folder.

Examining the C++ function ```AstroLib::teme_ecef``` shows that there are multiple extra inputs we require: 


In [5]:
# Test 2.1, Python + skyfield
from skyfield.framelib import itrs
itrf_pos, itrf_vel = pv.frame_xyz_and_velocity(itrs)
print(itrf_pos.km, itrf_vel.km_per_s)

[ 27797.22007278 -31465.7116508   -1428.37827126] [-0.02657353 -0.05888948  0.71167724]
